<a href="https://colab.research.google.com/github/aarongraybill/Math210/blob/main/ProblemSets/ProblemSet6/ProblemSet6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Math 210
## Aaron Graybill
### Problem Set 6
### 4/8/21



In [4]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

## Problem 1.


### a.
I convert the problem to a single source problem:

Edges: $\mathcal{E}=\left\{(0,1),(0,2),(0,4),(1,2),(1,3),(2,3),(2,4),(2,5),(3,4),(3,5),(4,5),(4,6),(5,4),(5,6)\right\}$.

Fix the flow on $(0,1),(0,2),(0,4)$ to $30,6,$ and $4$ respectively.

Let the currently unspecified constraint set be $\mathcal{C}$.

Maximize the flow into vertex $6$, so $\max_{x_{\mathcal{E}}\in\mathcal{C}}\left\{x_{46}+x_{56}\right\}$

We have the following constraint set:
\begin{align*}
x_{12}+x_{13}&=x_{01}\\
x_{23}+x_{24}+x_{25}&=x_{02}+x_{12}\\
x_{34}+x_{35}&=x_{13}+x_{23}\\
x_{45}+x_{46}&=x_{24}+x_{34}+x_{54}\\
x_{54}+x_{56}&=x_{25}+x_{35}+x_{45}\\
\end{align*}

In [5]:
a_eq=np.array([
            [1,0,-1,-1,-1,0,0,0,0,0,0],
            [0,1,1,0,0,-1,-1,0,0,0,0],
            [0,0,0,1,0,1,0,-1,-1,1,0],
            [0,0,0,0,1,0,1,1,0,-1,-1]])
print(a_eq)

b_eq=np.zeros((4))
print(b_eq)
a_leq=np.identity(11)
b_leq=[14,16,9,3,8,10,6,5,16,4,15]

c=[0,0,0,0,0,0,0,0,-1,0,-1]

linprog(c,A_ub=a_leq,b_ub=b_leq,A_eq=a_eq,b_eq=b_eq,method="simplex")


[[ 1  0 -1 -1 -1  0  0  0  0  0  0]
 [ 0  1  1  0  0 -1 -1  0  0  0  0]
 [ 0  0  0  1  0  1  0 -1 -1  1  0]
 [ 0  0  0  0  1  0  1  1  0 -1 -1]]
[0. 0. 0. 0.]


     con: array([0., 0., 0., 0.])
     fun: -27.0
 message: 'Optimization terminated successfully.'
     nit: 15
   slack: array([3., 0., 9., 0., 0., 0., 0., 4., 0., 0., 4.])
  status: 0
 success: True
       x: array([11., 16.,  0.,  3.,  8., 10.,  6.,  1., 16.,  4., 11.])

## Problem 3.


In [ ]:
#used to make label names:
  #source: geeksforgeeks.org/python-insert-the-string-at-the-beginning-of-all-items-in-a-list/
def prepend(list, str):   
    
  # Using '% s'
  str += '% s'
  list =  [str % i for i in list]
  return(list)

In [241]:
def create_eq_matrix(nvert,edgemat):

  A=np.zeros((nvert,(nvert**2)))  #too large will shrink later, 
                                  #but makes for easy indexing
  c=np.zeros(nvert**2) # ditto

  #loop over the edgemat, taking each path as an iterand
  for count, value in enumerate(edgemat[:,0:2]):

    col_num=(nvert)*(value[0])+(value[1]) # n columns from each vertex

    if (value[0]==0): #coming from source
      #Don't have to worry about constraint on sources
      A[value[1],col_num]=1
    elif (value[1]==(nvert-1)): #going to sink
      #dont have to worry about constraint on sink
      A[value[0],col_num]=-1
      c[col_num]=-1#Hey this is an important column, but it in the objective
    else: #intermediate
      A[value[1],col_num]=1
      A[value[0],col_num]=-1
  
  A=np.append(A,np.array([c]),axis=0) #add the row of cs

  #Name appropriately
  col_names=[] #create vector of names
  for i in itertools.product(np.arange(nvert),repeat=2): #all combos of indexes
    col_names += [str(i[0])+","+str(i[1])]
  row_names=prepend(np.arange(nvert),"V")
  row_names+="c" #add one final row name

  A=pd.DataFrame(A,index=row_names,columns=col_names)# put things into pd data

  # Remove unused rows and columns
  A=A.loc[(A != 0).any(axis=1), (A != 0).any(axis=0)]

  #separate A and c again
  c=A.iloc[-1:,:]
  A=A.iloc[:-1,:]

  #create easy matrices and vectors
  b_eq=np.zeros(A.shape[0])
  A_leq=np.identity(edgemat.shape[0])
  b_leq=edgemat[:,2]

  #do optimization
  optimal=linprog(c,A_eq=A,A_ub=A_leq,b_eq=b_eq,b_ub=b_leq,method="simplex")
  flows=np.array(np.array([optimal.x]))

  #generateoutput
  summary=pd.DataFrame(flows,index=["flow"],columns=A.columns)
  max="The maximum is: " +str(-optimal.fun)

  #print output and return
  print(max)
  return (summary,A)

### Problem 1 Confirmation

In [240]:
create_eq_matrix(6,edgemat=np.array([
                                  [0,1,14],
                                  [0,2,16],
                                  [1,2,9],
                                  [1,3,3],
                                  [1,4,8],
                                  [2,3,10],
                                  [2,4,6],
                                  [3,4,5],
                                  [3,5,16],
                                  [4,3,4],
                                  [4,5,15]]))[0]
            

The maximum is: 27.0


,"0,1","0,2","1,2","1,3","1,4","2,3","2,4","3,4","3,5","4,3","4,5"
flow,11.0,16.0,0.0,3.0,8.0,10.0,6.0,1.0,16.0,4.0,11.0


###  Problem 2 Confirmation.

In [239]:
create_eq_matrix(10,
                 np.array([
                           [0,1,6],
                           [0,5,6],
                           [0,8,2],
                           [1,2,3],
                           [1,5,3],
                           [2,3,2],
                           [2,5,6],
                           [2,7,3],
                           [3,4,4],
                           [4,7,4],
                           [4,9,7],
                           [5,6,3],
                           [5,8,2],
                           [6,7,2],
                           [6,8,1],
                           [7,9,3],
                           [8,9,6]
                           ]))[0]



The maximum is: 10.0


,"0,1","0,5","0,8","1,2","1,5","2,3","2,5","2,7","3,4","4,7","4,9","5,6","5,8","6,7","6,8","7,9","8,9"
flow,6.0,2.0,2.0,3.0,3.0,2.0,0.0,1.0,2.0,0.0,2.0,3.0,2.0,2.0,1.0,3.0,5.0


## Problem 4.


### a.
